In [1]:
import os 
import cv2 as cv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.image as mpimg
%matplotlib inline

from PIL import Image 

import tensorflow as tf

from keras.applications.resnet import ResNet50
from keras.models import Model,Sequential,load_model
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import Input,ZeroPadding2D,Activation,Add
from keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import load_img, img_to_array 
from keras.applications.resnet import decode_predictions
from keras.applications.resnet import preprocess_input


from keras.layers import GlobalAveragePooling2D
from keras.layers import AveragePooling1D 
from keras.layers import MaxPooling2D

from keras.initializers import glorot_uniform
import keras
import keras.utils
from keras import utils as np_utils

In [2]:
def create_dataset(img_folder):
    img_data=[]
    class_name=[]
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder,dir1)):
            img=cv.imread(os.path.join(img_folder,dir1,file),cv.COLOR_BGR2RGB)
            img=cv.resize(img,(224,224),interpolation=cv.INTER_AREA)
            img=img.astype('float32')
            img/=255
            img_data.append(img)
            class_name.append("COVID")
        break    
    return img_data,class_name
img_data_C,class_name_C=create_dataset(r"D:\Srikanth\CXR\Dataset1\COVID-19_Radiography_Dataset\COVID")
            

In [3]:
def create_dataset(img_folder):
    img_data=[]
    class_name=[]
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder,dir1)):
            img=cv.imread(os.path.join(img_folder,dir1,file),cv.COLOR_BGR2RGB)
            img=cv.resize(img,(224,224),interpolation=cv.INTER_AREA)
            img=img.astype('float32')
            img/=255
            img_data.append(img)
            class_name.append("Normal")
        break    
    return img_data,class_name
img_data_N,class_name_N=create_dataset(r"D:\Srikanth\CXR\Dataset1\COVID-19_Radiography_Dataset\Normal")
            

In [4]:
img_data_C.extend(img_data_N)

In [5]:
img_data_C[0].shape

(224, 224)

In [6]:
class_name_C.extend(class_name_N)

In [7]:
import numpy as np
target_dict={k:v for v,k in enumerate(np.unique(class_name_C))}
target_dict

{'COVID': 0, 'Normal': 1}

In [8]:
target_val=[target_dict[class_name_C[i]] for i in range(len(class_name_C))]

In [9]:
len(target_val)

13808

In [10]:
opt=tf.keras.optimizers.Adam(learning_rate=0.001)

In [11]:
x=np.array(tf.expand_dims(img_data_C,axis=-1), np.float32)
y=np.array(list(map(int,target_val)), np.float32)

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,train_size=0.75)

In [13]:
print(x_train[0][0][0])

[0.68235296]


In [14]:
print(x_train.shape)

(10356, 224, 224, 1)


In [15]:
print(x_test[0][0][0])

[0.8745098]


In [16]:
print(x_test.shape)

(3452, 224, 224, 1)


In [17]:
es=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [18]:
from keras.layers import MaxPooling2D,AveragePooling2D

In [19]:
model=Sequential()
model.add(Conv2D(64, kernel_size=(7,7),strides=(2,2) ,activation= 'relu', input_shape=(224,224,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256,(1,1), activation= 'relu'))
model.add(Conv2D(64,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(64,(3,3), activation= 'relu',strides=(1,1)))
model.add(Conv2D(256,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(64,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(64,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(256,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(64,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(64,(3,3), activation= 'relu',strides=(1,1)))
model.add(Conv2D(256,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(512,(1,1), activation= 'relu',strides=(2,2)))
model.add(Conv2D(128,(1,1), activation= 'relu',strides=(2,2)))
model.add(Conv2D(128,(3,3), activation= 'relu',strides=(1,1)))
model.add(Conv2D(512,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(128,(1,1), activation= 'relu',strides=(1,1)))
model.add(Conv2D(128,(3,3), activation= 'relu',strides=(1,1)))
model.add(Conv2D(512,(1,1), activation= 'relu',strides=(1,1)))


model.add(AveragePooling2D(pool_size=(7,7)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy, optimizer=opt, metrics=['accuracy'])
model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=100,epochs=20,callbacks=es)

Epoch 1/20
104/104 [==============================] - 332s 3s/step - loss: 0.5847 - accuracy: 0.7388 - val_loss: 0.5814 - val_accuracy: 0.7361
Epoch 2/20
104/104 [==============================] - 324s 3s/step - loss: 0.5763 - accuracy: 0.7388 - val_loss: 0.5762 - val_accuracy: 0.7361
Epoch 3/20
104/104 [==============================] - 316s 3s/step - loss: 0.5584 - accuracy: 0.7392 - val_loss: 0.5267 - val_accuracy: 0.7375
Epoch 4/20
104/104 [==============================] - 314s 3s/step - loss: 0.5166 - accuracy: 0.7594 - val_loss: 0.4976 - val_accuracy: 0.7775
Epoch 5/20
104/104 [==============================] - 313s 3s/step - loss: 0.4648 - accuracy: 0.7863 - val_loss: 0.4182 - val_accuracy: 0.8019
Epoch 6/20
104/104 [==============================] - 314s 3s/step - loss: 0.4002 - accuracy: 0.8217 - val_loss: 0.3679 - val_accuracy: 0.8384
Epoch 7/20
104/104 [==============================] - 316s 3s/step - loss: 0.3736 - accuracy: 0.8350 - val_loss: 0.3514 - val_accuracy: 0.8401

In [20]:
import sklearn
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier  
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier

In [21]:
print(x_test.shape)

(3452, 224, 224, 1)


In [22]:
def metrics(y_test,pred):
    
    scores={}
    #Accuracy
    from sklearn.metrics import accuracy_score
    scores["accuracy"]=accuracy_score(y_test,pred)

    # Recall
    from sklearn.metrics import recall_score
    scores["recall"]=recall_score(y_test, pred)
 
    # Precision
    from sklearn.metrics import precision_score
    scores["precision"]=precision_score(y_test, pred)

    #F1 score
    from sklearn.metrics import f1_score
    scores["f1_score"]=f1_score(y_test,pred)

    #Calculate AUC 
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, treshold = roc_curve(y_test, pred)
    scores["roc"]=roc_auc = auc(fpr, tpr)
    return scores

In [26]:
x_train.shape

(10356, 224, 224, 1)

In [27]:
model.evaluate(x_train,y_train)

324/324 [==============================] - 87s 269ms/step - loss: 0.1520 - accuracy: 0.9392


[0.15199628472328186, 0.9391657114028931]

In [28]:
model.evaluate(x_test,y_test)

108/108 [==============================] - 29s 270ms/step - loss: 0.2414 - accuracy: 0.9059


[0.24139820039272308, 0.9058516621589661]

In [29]:
y_pred=model.predict(x_test)
y_pred

108/108 [==============================] - 29s 270ms/step


array([[0.99715126],
       [0.9996337 ],
       [0.39139825],
       ...,
       [0.9894812 ],
       [0.99861276],
       [0.00682746]], dtype=float32)

In [31]:
print(metrics(y_test,y_pred.round())) 

{'accuracy': 0.9058516801853997, 'recall': 0.9673356946084218, 'precision': 0.9103703703703704, 'f1_score': 0.9379889334096546, 'roc': 0.8508467715632668}


In [1]:
from keras.models import load_model
model.save("resnet18org.h5")

NameError: name 'model' is not defined

In [ ]:
feature_extractor = keras.Model(
    inputs=model.inputs,
    outputs=model.get_layer(name="2048").output,)

In [ ]:
f=open("feature.csv","w")
for i in range(len(img_datac)):
    img=img_datac[i]
    img=np.expand_dims(img,axis=0)
    features =list( feature_extractor(img))
    for j in list(features[0]):
        f.write(str(float(j))+",")
    f.write(str(target_val[i])+"\n")    
f.close()
    